In [1]:
import pandas as pd
import requests
import re
from datetime import datetime, timedelta

def find_working_london_dataset(max_months=12):
    """
    Parcourt les archives d'Inside Airbnb pour trouver un dataset de Londres 
    valide et non vide.
    """
    # On commence la recherche à partir d'aujourd'hui
    current_date = datetime.now().replace(day=1)
    
    print("🔍 Recherche d'un dataset Londres valide (avec prix et équipements)...")

    for i in range(max_months):
        # Format de date typique Inside Airbnb (souvent entre le 5 et le 15 du mois)
        # On teste le 6 du mois, date fréquente de mise à jour
        date_str = current_date.strftime("%Y-%m-06")
        url = f"http://data.insideairbnb.com/united-kingdom/england/london/{date_str}/data/listings.csv.gz"
        
        try:
            # 1. Vérification de l'existence du fichier (HEAD)
            resp = requests.head(url, timeout=5)
            if resp.status_code == 200:
                # 2. Chargement d'un échantillon pour vérifier le contenu
                # On charge 500 lignes pour être sûr d'avoir un échantillon représentatif
                df_test = pd.read_csv(url, nrows=500, compression='gzip', low_memory=False)
                
                # 3. Vérification des colonnes et des données NA
                has_cols = 'price' in df_test.columns and 'amenities' in df_test.columns
                
                if has_cols:
                    # Vérification que les prix ne sont pas TOUS de type NA
                    # On compte les valeurs non-nulles
                    valid_prices = df_test['price'].notna().sum()
                    
                    if valid_prices > 0:
                        print(f"🎯 TROUVÉ ! Date : {date_str}")
                        print(f"📊 Échantillon : {valid_prices}/500 prix valides détectés.")
                        return url
                    else:
                        print(f"⚠️ Date {date_str} ignorée : La colonne 'price' est vide (que des NA).")
                else:
                    print(f"⚠️ Date {date_str} ignorée : Colonnes manquantes.")
            
        except Exception:
            pass # On continue si l'URL exacte n'existe pas
        
        # On recule d'un mois pour l'itération suivante
        current_date = (current_date - timedelta(days=1)).replace(day=1)

    return None

# ==============================================================================
# EXÉCUTION & CHARGEMENT
# ==============================================================================
london_url = find_working_london_dataset()

if london_url:
    # On ne charge que les colonnes nécessaires pour économiser la RAM
    cols_to_keep = ['id', 'amenities', 'price', 'latitude', 'longitude', 
                    'property_type', 'room_type', 'accommodates', 'bedrooms', 
                    'beds', 'bathrooms_text', 'neighbourhood_cleansed']
    
    print("⏳ Téléchargement du dataset complet...")
    df_london_raw = pd.read_csv(london_url, usecols=cols_to_keep, compression='gzip', low_memory=False)
    
    # Nettoyage immédiat des lignes où le prix est NA (pour être 100% sûr)
    df_london_raw = df_london_raw.dropna(subset=['price'])
    
    print(f"✅ Londres chargé avec succès : {len(df_london_raw)} lignes sans NA sur le prix.")
else:
    print("❌ Impossible de trouver un dataset valide automatiquement.")

🔍 Recherche d'un dataset Londres valide (avec prix et équipements)...
🎯 TROUVÉ ! Date : 2025-05-06
📊 Échantillon : 317/500 prix valides détectés.
⏳ Téléchargement du dataset complet...
✅ Londres chargé avec succès : 61964 lignes sans NA sur le prix.
